# Solar Panel Permits Analysis

This notebook provides a detailed analysis of solar panel installation permits in the CCS permits dataset, examining trends, installers, locations, and system characteristics.

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Load the dataset
df = pd.read_csv('source/CCS_Permits.csv')
print(f"Total permits in dataset: {len(df):,}")

# Convert date columns
df['issueDate'] = pd.to_datetime(df['issueDate'], errors='coerce')
df['completeDate'] = pd.to_datetime(df['completeDate'], errors='coerce')

## 2. Identifying Solar Permits

In [ ]:
# Search for solar-related permits using multiple keywords
solar_keywords = ['solar', 'Solar', 'SOLAR', 'photovoltaic', 'PV', 'panel']
solar_mask = df['comments'].str.contains('|'.join(solar_keywords), case=False, na=False)
solar_permits = df[solar_mask].copy()

print(f"Total number of solar permits found: {len(solar_permits):,}")
print(f"Percentage of total permits: {len(solar_permits)/len(df)*100:.2f}%")
print(f"\nDate range of solar permits: {solar_permits['issueDate'].min()} to {solar_permits['issueDate'].max()}")

In [ ]:
# Display sample solar permit comments
print("Sample solar permit descriptions:")
print("=" * 80)
for i, (idx, row) in enumerate(solar_permits.head(5).iterrows()):
    print(f"\n{i+1}. Permit #{row['permitNumber']}")
    print(f"   Address: {row['Display']}")
    print(f"   Comment: {row['comments'][:200]}...")

## 3. Temporal Analysis of Solar Permits

In [ ]:
# Extract year and month from issue dates
solar_permits['issue_year'] = solar_permits['issueDate'].dt.year
solar_permits['issue_month'] = solar_permits['issueDate'].dt.month
solar_permits['issue_quarter'] = solar_permits['issueDate'].dt.quarter

# Analyze solar permits by year
solar_by_year = solar_permits['issue_year'].value_counts().sort_index()
print("Solar permits by year:")
print(solar_by_year)

In [ ]:
# Create comprehensive temporal visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Yearly trend
solar_by_year.plot(kind='line', ax=axes[0, 0], marker='o', linewidth=2, markersize=8, color='orange')
axes[0, 0].set_title('Solar Permits by Year', fontsize=14)
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Number of Permits')
axes[0, 0].grid(True, alpha=0.3)

# Monthly distribution
monthly_solar = solar_permits['issue_month'].value_counts().sort_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[0, 1].bar(monthly_solar.index, monthly_solar.values, color='skyblue')
axes[0, 1].set_title('Solar Permits by Month (All Years)', fontsize=14)
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Number of Permits')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].set_xticklabels(month_names, rotation=45)

# Quarterly trend
quarterly_solar = solar_permits.groupby(['issue_year', 'issue_quarter']).size().reset_index(name='count')
pivot_quarterly = quarterly_solar.pivot(index='issue_year', columns='issue_quarter', values='count')
pivot_quarterly.plot(kind='bar', ax=axes[1, 0], stacked=True)
axes[1, 0].set_title('Solar Permits by Quarter', fontsize=14)
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Number of Permits')
axes[1, 0].legend(title='Quarter', labels=['Q1', 'Q2', 'Q3', 'Q4'])

# Growth rate
growth_rate = solar_by_year.pct_change() * 100
growth_rate.dropna().plot(kind='bar', ax=axes[1, 1], color='green')
axes[1, 1].set_title('Year-over-Year Growth Rate (%)', fontsize=14)
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Growth Rate (%)')
axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.suptitle('Temporal Analysis of Solar Permits', fontsize=16)
plt.tight_layout()
plt.show()

## 4. Solar System Capacity Analysis

In [ ]:
# Extract capacity information from comments
def extract_capacity(comment):
    if pd.isna(comment):
        return None
    # Look for patterns like "5.2 kW", "5.2kW", "5.2 KW", etc.
    patterns = [
        r'(\d+\.?\d*)\s*[kK][wW]',
        r'CAPACITY\s*_*(\d+\.?\d*)_*\s*[kK][wW]',
        r'CAPACITY\s*(\d+\.?\d*)\s*[kK][wW]'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, comment, re.IGNORECASE)
        if match:
            return float(match.group(1))
    return None

solar_permits['capacity_kw'] = solar_permits['comments'].apply(extract_capacity)
valid_capacity = solar_permits[solar_permits['capacity_kw'].notna()]

print(f"Permits with extractable capacity: {len(valid_capacity)} ({len(valid_capacity)/len(solar_permits)*100:.1f}%)")
print(f"\nCapacity Statistics:")
print(valid_capacity['capacity_kw'].describe())

In [ ]:
# Visualize capacity distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram of capacities
valid_capacity['capacity_kw'].hist(bins=30, ax=ax1, edgecolor='black', alpha=0.7)
ax1.set_title('Distribution of Solar System Capacities', fontsize=14)
ax1.set_xlabel('Capacity (kW)')
ax1.set_ylabel('Frequency')
ax1.axvline(valid_capacity['capacity_kw'].median(), color='red', linestyle='dashed', linewidth=2, label=f'Median: {valid_capacity["capacity_kw"].median():.2f} kW')
ax1.legend()

# Box plot by year
valid_capacity_yearly = valid_capacity[valid_capacity['issue_year'] >= 2015]
valid_capacity_yearly.boxplot(column='capacity_kw', by='issue_year', ax=ax2)
ax2.set_title('Solar System Capacity by Year', fontsize=14)
ax2.set_xlabel('Year')
ax2.set_ylabel('Capacity (kW)')
ax2.get_figure().suptitle('')

plt.tight_layout()
plt.show()

## 5. Top Solar Installers Analysis

In [ ]:
# Analyze top solar installers
solar_installers = solar_permits['applicantName'].value_counts().head(15)
print("Top 15 Solar Installers/Applicants:")
print("=" * 50)
for i, (installer, count) in enumerate(solar_installers.items(), 1):
    print(f"{i:2d}. {installer:<40} {count:>4} permits")

In [ ]:
# Visualize top installers
plt.figure(figsize=(12, 8))
solar_installers.plot(kind='barh', color='coral')
plt.title('Top 15 Solar Installers by Number of Permits', fontsize=16)
plt.xlabel('Number of Permits', fontsize=12)
plt.ylabel('Installer/Applicant', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze market share of top installers
top_5_installers = solar_installers.head(5)
other_count = len(solar_permits) - top_5_installers.sum()
market_share = pd.Series({
    **dict(top_5_installers),
    'Others': other_count
})

plt.figure(figsize=(10, 8))
colors = plt.cm.Set3(range(len(market_share)))
plt.pie(market_share.values, labels=market_share.index, autopct='%1.1f%%', startangle=90, colors=colors)
plt.title('Market Share of Solar Installers', fontsize=16)
plt.axis('equal')
plt.show()

## 6. Geographic Distribution of Solar Permits

In [ ]:
# Analyze solar permits by neighborhood
solar_neighborhoods = solar_permits['Neighborhoods_Desc'].value_counts().head(20)
print(f"Solar permits are distributed across {solar_permits['Neighborhoods_Desc'].nunique()} neighborhoods")
print("\nTop 20 Neighborhoods for Solar Installations:")
print(solar_neighborhoods)

In [ ]:
# Visualize neighborhood distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Bar chart of top neighborhoods
solar_neighborhoods.head(10).plot(kind='barh', ax=ax1, color='lightgreen')
ax1.set_title('Top 10 Neighborhoods by Solar Permits', fontsize=14)
ax1.set_xlabel('Number of Solar Permits')

# Ward distribution
ward_solar = solar_permits['Wards'].value_counts().sort_index()
ward_solar.plot(kind='bar', ax=ax2, color='lightblue')
ax2.set_title('Solar Permits by Ward', fontsize=14)
ax2.set_xlabel('Ward')
ax2.set_ylabel('Number of Solar Permits')

plt.tight_layout()
plt.show()

## 7. Solar Permit Status and Duration Analysis

In [ ]:
# Analyze permit status
solar_status = solar_permits['status'].value_counts()
print("Solar Permit Status Distribution:")
print(solar_status)
print(f"\nCompletion rate: {(solar_status.get('Closed', 0) / len(solar_permits) * 100):.1f}%")

In [ ]:
# Calculate permit duration for completed permits
solar_permits['duration_days'] = (solar_permits['completeDate'] - solar_permits['issueDate']).dt.days
completed_solar = solar_permits[(solar_permits['status'] == 'Closed') & (solar_permits['duration_days'] > 0) & (solar_permits['duration_days'] < 1000)]

print("Solar Permit Duration Statistics (in days):")
print(completed_solar['duration_days'].describe())

# Visualize duration
plt.figure(figsize=(10, 6))
completed_solar['duration_days'].hist(bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Solar Permit Duration', fontsize=16)
plt.xlabel('Duration (days)')
plt.ylabel('Frequency')
plt.axvline(completed_solar['duration_days'].median(), color='red', linestyle='dashed', linewidth=2, label=f'Median: {completed_solar["duration_days"].median():.0f} days')
plt.legend()
plt.tight_layout()
plt.show()

## 8. Financial Analysis of Solar Permits

In [ ]:
# Analyze permit values and fees
solar_permits['value'] = pd.to_numeric(solar_permits['value'], errors='coerce')
solar_permits['totalFees'] = pd.to_numeric(solar_permits['totalFees'], errors='coerce')

valid_values = solar_permits[solar_permits['value'] > 0]
print("Solar Permit Value Statistics:")
print(valid_values['value'].describe())
print(f"\nTotal value of solar permits: ${valid_values['value'].sum():,.2f}")
print(f"Average value per permit: ${valid_values['value'].mean():,.2f}")

In [ ]:
# Analyze relationship between capacity and value
capacity_value = solar_permits[(solar_permits['capacity_kw'].notna()) & (solar_permits['value'] > 0)]

plt.figure(figsize=(10, 6))
plt.scatter(capacity_value['capacity_kw'], capacity_value['value'], alpha=0.6)
plt.xlabel('System Capacity (kW)', fontsize=12)
plt.ylabel('Permit Value ($)', fontsize=12)
plt.title('Solar System Capacity vs Permit Value', fontsize=16)

# Add trend line
if len(capacity_value) > 10:
    z = np.polyfit(capacity_value['capacity_kw'], capacity_value['value'], 1)
    p = np.poly1d(z)
    plt.plot(capacity_value['capacity_kw'].sort_values(), p(capacity_value['capacity_kw'].sort_values()), "r--", alpha=0.8, label=f'Trend: ${z[0]:.0f}/kW')
    plt.legend()

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Seasonal Patterns in Solar Installations

In [ ]:
# Create a heatmap of solar permits by year and month
solar_heatmap_data = solar_permits.pivot_table(values='permitNumber', index='issue_year', columns='issue_month', aggfunc='count', fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(solar_heatmap_data, cmap='YlOrRd', annot=True, fmt='d', cbar_kws={'label': 'Number of Solar Permits'})
plt.title('Heatmap of Solar Permits by Year and Month', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Year', fontsize=12)
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
plt.xticks(range(12), month_labels)
plt.tight_layout()
plt.show()

## 10. Summary and Key Insights

In [ ]:
# Generate comprehensive summary
print("=" * 80)
print("SOLAR PERMITS ANALYSIS SUMMARY")
print("=" * 80)
print(f"\nTotal solar permits: {len(solar_permits):,}")
print(f"Percentage of all permits: {len(solar_permits)/len(df)*100:.2f}%")
print(f"Date range: {solar_permits['issueDate'].min().strftime('%Y-%m-%d')} to {solar_permits['issueDate'].max().strftime('%Y-%m-%d')}")

print(f"\nTop installer: {solar_installers.index[0]} ({solar_installers.iloc[0]} permits)")
print(f"Most active neighborhood: {solar_neighborhoods.index[0]} ({solar_neighborhoods.iloc[0]} permits)")

if len(valid_capacity) > 0:
    print(f"\nCapacity Statistics:")
    print(f"  - Average system size: {valid_capacity['capacity_kw'].mean():.2f} kW")
    print(f"  - Median system size: {valid_capacity['capacity_kw'].median():.2f} kW")
    print(f"  - Total installed capacity: {valid_capacity['capacity_kw'].sum():,.0f} kW")

if len(valid_values) > 0:
    print(f"\nFinancial Statistics:")
    print(f"  - Total permit value: ${valid_values['value'].sum():,.2f}")
    print(f"  - Average permit value: ${valid_values['value'].mean():,.2f}")

print(f"\nPermit completion rate: {(solar_status.get('Closed', 0) / len(solar_permits) * 100):.1f}%")
if len(completed_solar) > 0:
    print(f"Average permit duration: {completed_solar['duration_days'].mean():.0f} days")

# Year-over-year growth
recent_years = solar_by_year[solar_by_year.index >= 2020]
if len(recent_years) > 1:
    growth_2020_2023 = ((recent_years.iloc[-1] - recent_years.iloc[0]) / recent_years.iloc[0] * 100)
    print(f"\nGrowth from {recent_years.index[0]} to {recent_years.index[-1]}: {growth_2020_2023:.1f}%")

print("\n" + "=" * 80)

In [ ]:
# Create a summary dashboard
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Yearly trend
solar_by_year.plot(kind='line', ax=axes[0, 0], marker='o', color='orange', linewidth=2, markersize=8)
axes[0, 0].set_title('Solar Permits by Year', fontsize=14)
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Number of Permits')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Top installers
solar_installers.head(5).plot(kind='bar', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Top 5 Solar Installers', fontsize=14)
axes[0, 1].set_xlabel('Installer')
axes[0, 1].set_ylabel('Number of Permits')
axes[0, 1].tick_params(axis='x', rotation=45)

# Plot 3: Capacity distribution
if len(valid_capacity) > 0:
    valid_capacity['capacity_kw'].hist(bins=20, ax=axes[1, 0], edgecolor='black', alpha=0.7, color='lightgreen')
    axes[1, 0].set_title('Distribution of System Capacities', fontsize=14)
    axes[1, 0].set_xlabel('Capacity (kW)')
    axes[1, 0].set_ylabel('Frequency')
else:
    axes[1, 0].text(0.5, 0.5, 'No capacity data available', ha='center', va='center', transform=axes[1, 0].transAxes)

# Plot 4: Monthly seasonality
monthly_avg = solar_permits.groupby('issue_month').size()
axes[1, 1].bar(monthly_avg.index, monthly_avg.values, color='skyblue')
axes[1, 1].set_title('Solar Permits by Month (All Years)', fontsize=14)
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('Total Permits')
axes[1, 1].set_xticks(range(1, 13))
axes[1, 1].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)

plt.suptitle('Solar Permits Analysis Dashboard', fontsize=18, y=1.02)
plt.tight_layout()
plt.show()